# Hotspot calculation demo


This notebook detects and classifies hotspots of the von Mises stress in a connected FEM mesh. Each element/node entry of the mesh receives a number of the hotspot it is member of. "0" means the element/node is not part of any hotspots. "1" means that the element/node is part of the hotspot with the highes peak, "2" the same for the second highest peak and so forth.

See [module documentation](https://pylife.readthedocs.io/en/latest/mesh/hotspot.html) further details.

In [ ]:
import numpy as np
import pylife
import pandas as pd
import scipy.stats as stats
import pylife.stress.equistress
import pylife.strength.meanstress
import pylife.mesh.meshsignal
import pylife.mesh.hotspot
import pylife.vmap

import pyvista as pv

In [ ]:
vm_mesh = pylife.vmap.VMAPImport("two_notched_beam.vmap")
mesh = (vm_mesh.make_mesh('beam', 'STATE-1')
        .join_coordinates()
        .join_variable('STRESS_CAUCHY')
        .join_variable('DISPLACEMENT')
        .to_frame())

## Equivalent stress calculation

In [ ]:
mesh['mises'] = mesh.equistress.mises()

## Hot spot Calculation ##

In [ ]:
threshold = .3 # factor of the maximum local value
mesh['hotspot'] = mesh.hotspot.calc("mises", threshold)
display(mesh[['x', 'y', 'z', 'mises', 'hotspot']])

In [ ]:
print("%d hotspots found over the threshold" % mesh['hotspot'].max())

In [ ]:
grid = pv.UnstructuredGrid(*mesh.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=mesh.groupby('element_id')['hotspot'].first().to_numpy(), show_edges=True)
plotter.add_scalar_bar()
plotter.show()

### Hotspots

In [ ]:
first_hotspot = mesh[mesh['hotspot'] == 1]
display(first_hotspot)

In [ ]:
second_hotspot = mesh[mesh['hotspot'] == 2]
display(second_hotspot)

In [ ]:
third_hotspot = mesh[mesh['hotspot'] == 3]
display(third_hotspot)